# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [133]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [134]:
# Realizamos uma solicitação GET para a API do Tesouro Nacional a fim de obter os dados dos municípios

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url, timeout=120)

In [135]:
# Obtemos o código de status da resposta da requisição

r.status_code

200

In [136]:
# Convertemos a resposta da requisição para JSON

data_json = r.json()

In [137]:
# Criamos um DataFrame a partir dos dados da chave 'items' do JSON
# Filtramos os dados do DataFrame para selecionar apenas os registros relacionados aos municípios do estado do Rio de Janeiro

df = pd.DataFrame(data_json['items'])
rio = df[(df['tipo_interessado'] == 'Município') & (df['uf'] == 'RJ')]

In [138]:
# 2) 
# Contamos e seleciona os 3 principais valores da coluna 'status' no DataFrame 'rio'
# Imprimos os 3 principais valores e suas contagens

top3_status = rio['status'].value_counts().head(3)
print(top3_status)

status
Deferido        96
Arquivado       53
Regularizado    26
Name: count, dtype: int64


In [139]:
# 3) 
# Extraimos o ano da coluna 'data_status' e adiciona como uma nova coluna 'ano_status' no DataFrame 'rio'

rio['ano_status'] = rio['data_status'].str.slice(-4)

In [140]:
# 4) 
# Calculamos a contagem de frequência dos anos presentes na coluna 'ano_status' do DataFrame 'rio'
# Imprimos a contagem de frequência dos anos

frequencia_ano = rio['ano_status'].value_counts()
print(frequencia_ano)


ano_status
2008    37
2007    29
2014    20
2010    19
2019    17
2013    15
2020    15
2012    13
2022    11
2023    11
2011    10
2009     9
2015     8
2017     5
2021     5
2016     4
2006     4
2018     4
2002     4
2004     1
2003     1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [141]:
#1) 
# Construimos a URL da consulta com base nos parâmetros UF e tipo_interessado
# Fazemos uma solicitação GET à API utilizando a URL
# Verificamos se houve algum erro na resposta
# Convertemos o JSON retornado para um DataFrame 
# Utilizamos a chave 'items' do JSON, se existir, caso contrário retorna uma lista vazia

def consulta(uf, tipo_interessado):
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    resposta = requests.get(url, timeout=120)
    resposta.raise_for_status()
    return pd.DataFrame(resposta.json().get('items', []))


In [142]:
# 2) 
# Realizamos uma consulta para o estado de Minas Gerais com interessado igual a 'Estado'
# Filtramos o DataFrame para selecionar as solicitações com status igual a 'Arquivado por decurso de prazo'
# Obtemos o número de solicitações arquivadas contando o número de linhas no DataFrame filtrado
# Imprimimos o número de solicitações arquivadas para o estado de Minas Gerais

minas = consulta('MG', 'Estado')
solicitacoes = minas[minas['status'] == 'Arquivado por decurso de prazo'].shape[0]
print(solicitacoes)


1


In [143]:
# 3) 
# Realizamos uma consulta para os municípios da Bahia com interessado igual a 'Município'
# Filtramos o DataFrame para selecionar somente os registros com UF igual a 'BA'
# Verificamos se há solicitações deferidas no DataFrame filtrado
# Filtramos o DataFrame para selecionar somente os registros com status igual a 'Deferido'
# Obtemos o município com mais solicitações deferidas

bahia_municipio = consulta('BA', 'Município')
bahia_municipio = bahia_municipio[bahia_municipio['uf'] == 'BA']

if 'Deferido' in bahia_municipio['status'].values:
    municipio_deferido = bahia_municipio[bahia_municipio['status'] == 'Deferido']['interessado'].value_counts().idxmax()
    print('O município da Bahia com mais solicitações deferidas é:', municipio_deferido)
else:
    print('Não há solicitações deferidas para municípios da Bahia.')


O município da Bahia com mais solicitações deferidas é: Luís Eduardo Magalhães


In [144]:
# 4) 
# Realizamos uma consulta para o estado da Bahia com interessado igual a 'Estado'
# Salvamos os dados do estado da Bahia em um arquivo CSV chamado 'solicitacoes_bahia.csv'
# Imprimimos os dados do estado da Bahia

bahia_estado = consulta('BA', 'Estado')
bahia_estado.to_csv('solicitacoes_bahia.csv')
print(bahia_estado)

    id_pleito tipo_interessado interessado  cod_ibge  uf  \
0       10431           Estado       Bahia        29  BA   
1       42037           Estado       Bahia        29  BA   
2       60782           Estado       Bahia        29  BA   
3       64270           Estado       Bahia        29  BA   
4       18495           Estado       Bahia        29  BA   
..        ...              ...         ...       ...  ..   
79      18476           Estado       Bahia        29  BA   
80      18503           Estado       Bahia        29  BA   
81      18499           Estado       Bahia        29  BA   
82      18507           Estado       Bahia        29  BA   
83      18523           Estado       Bahia        29  BA   

                 num_pvl                                             status  \
0                   None  Encaminhado à PGFN com manifestação técnica fa...   
1   PVL02.007745/2019-06                                           Deferido   
2   PVL02.007004/2022-12                  